### Cas Kaggke: Topic Labeled News Dataset

En aquest document veurem, analitzarem i compararem diferents classificadors sobre una base de dades de Kaggle. <br/><br/>
Els objectius principals son:
- Analitzar la base de dades
- Aplicar diferents classificadors 
- Analitzar els resultats obtinguts dels classificadors
- Extreure conclusions

La base de dades de kaggle utilitzada és la de "Topic Labeled News Dataset". La base de dades és una recopilació d'articles de notícies. A continucació començarem veient i analitzant la base de dades.

In [10]:
# Importem llibreries
import ipywidgets as widgets
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np #importem la llibreria
import random

from gensim.parsing.preprocessing import remove_stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [11]:
# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path,sep = None,engine='python')
    return dataset

# Carreguem dataset asignat
dataset = load_dataset('labelled_newscatcher_dataset.csv')
data = dataset.values

In [12]:
dataset.head()

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en


In [13]:
dataset.describe()

,topic,link,domain,published_date,title,lang
count,108774,108774,108774,108774,108774,108774
unique,8,106130,5164,68743,103180,1
top,ENTERTAINMENT,https://www.google.com/,dailymail.co.uk,2020-08-04 01:00:00,"US tops 5 million confirmed virus cases, to Eu...",en
freq,15000,19,1855,41,21,108774


Com es pot observar, la BD està formada per 6 atributs:
- **topic:** topic en el que s'ha classificat l'article, segons el Kaggle n'hi han 8 tipos(BUSINESS, ENTERTAINMENT, HEALTH, NATION, SCIENCE, SPORTS, TECHNOLOGY i WORLD)
- **link:** link d'on trobar l'article
- **domain:** domini de la pàgina en que va ser publicat l'article
- **published_date:** data en que l'article va ser publicat
- **title:** títol de l'article
- **lang:** llengua en el que està escrit l'article, podem observar com tots els articles estàn en Anglés

Ara continuem analitzant la base de dades.

In [14]:
print(dataset.dtypes)

topic             object
link              object
domain            object
published_date    object
title             object
lang              object
dtype: object


In [15]:
print(dataset.isnull().sum())

topic             0
link              0
domain            0
published_date    0
title             0
lang              0
dtype: int64


In [16]:
print (dataset['published_date'].min())
print (dataset['published_date'].max())

2012-09-16 04:44:50
2020-08-18 05:49:00


In [17]:
contTopic = dataset['topic'].value_counts()
contTopic

ENTERTAINMENT    15000
HEALTH           15000
SPORTS           15000
TECHNOLOGY       15000
WORLD            15000
NATION           15000
BUSINESS         15000
SCIENCE           3774
Name: topic, dtype: int64

Observacions importants:
- no tenim nungún atribut numéric
- no tenim ninguna entrada / fila en la base de dades amb algún valor a null
- podem observar com hi han 8 topics diferents, 7 d'ells consten de 15000 entrades en la BD i un d'ells (el de SCIENCE) 3774
- la data de publicació dels articles més antiga és del 2012-09-16 i la més recent del 2020-08-18

He decidit entrenar i comparar classificadors que a partir del títol (camp "title" en la BD) faci una classificació d'un article segons el topic (camp objectiu "topic").
Descarto la llengua ja que tots estàn escrits en la mateixa llengua, el link, el domini i la data de publicació perque no aportarien ninguna informació sobre el tópic de l'article. 

A continuació, entrenarem un classificador Naive Bayes. 

In [33]:
from sklearn.pipeline import Pipeline

X = dataset['title']
Y = dataset['topic'] #objectiu

particions = [0.1, 0.2, 0.4] #porcentatge de dades que s'utilitzaran en test

for part in particions:
    # Dividim dades en part d'entrenament i test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=part, random_state=0)

    #Creem un pipeline, que serveix per aplicar seqüencialment una llista de transformades i un 
    #estimador final (l'estimador final ha de portar implementat "fit").
    #
    #La funció de "CountVectorizer" s'utilitza per convertir un text en un vector de recomptes, es a dir, 
    #retorna un diccionari de paraules i el seu recompte de vagades que ha sortit cada paraula.
    #
    #La funció de "TfidfTransformer" transforma el diccionari / recompte de la funció "CountVectorizer"
    #en una representació normalitzada tf (frequéncia) o tf-idf (frequéncia multiplicada per la inversa de la frequéncia),
    #aixó fa que obtinguem la frequéncia d'aparició d'una paraula.
    #
    #Finalment, tenim la funció "MultinomialNB" que és el classificador multinomial Naive Bayes, s'utilitza aquest perque 
    #és adequat per a la classificació amb característiques discretes (p. ex., recompte de paraules per a la classificació 
    #de text)
    
    naiveBayes = Pipeline([('countVect', CountVectorizer()),
                          ('tfidfTransf', TfidfTransformer()),
                          ('clf', MultinomialNB()),
    ])
    
    #entrenem
    naiveBayes = naiveBayes.fit(x_train, y_train)

    #test
    predicted = naiveBayes.predict(x_test)
    print ("Correct classification Naive Bayes with stopwords      ", part, "% of the data: ", np.mean(predicted == y_test))


Correct classification Naive Bayes with stopwords       0.1 % of the data:  0.7868174296745726
Correct classification Naive Bayes with stopwords       0.2 % of the data:  0.7875890599862101
Correct classification Naive Bayes with stopwords       0.4 % of the data:  0.7785796368650885


Com podem observar, el millor accuracy que s'ha obtingut ha sigut l'entrenat amb el 80% de les dades amb un accuracy del 78,75%.  <br/><br/>
Cal mencionar que tots els títols dels articles, podien contenir paraules que no aportaven cap informació útil per decidir en quina categoria s'havia de classificar un text ("stopwowrds") com per exemple articles, preposicions, etc...
Així que per comprovar si aquestes paraules tenien alguna influencia alhora de classificar els articles, les eliminarem i tornarem a entrenar el model. Per a fer-ho s'utilitza el métode "remove_stopwords" de la llibreria "gensim.parsing.preprocessing".

In [30]:
#eliminem els stopwords
dataTitleSW = dataset['title'].apply(lambda x: remove_stopwords(x))

In [31]:
#mirem si ha cannviat
dataTitleSW.head()

0    A closer look water-splitting's solar fuel pot...
1    An irresistible scent makes locusts swarm, stu...
2    Artificial intelligence warning: AI know bette...
3     Glaciers Could Have Sculpted Mars Valleys: Study
4    Perseid meteor shower 2020: What time huge bri...
Name: title, dtype: object

In [35]:
from sklearn.pipeline import Pipeline

X = dataTitleSW
Y = dataset['topic'] #objectiu

particions = [0.1, 0.2, 0.4] #porcentatge de dades que s'utilitzaran en test

for part in particions:
    # Dividim dades en part d'entrenament i test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=part, random_state=0)

    #Creem un pipeline, que serveix per aplicar seqüencialment una llista de transformades i un 
    #estimador final (l'estimador final ha de portar implementat "fit").
    #
    #La funció de "CountVectorizer" s'utilitza per convertir un text en un vector de recomptes, es a dir, 
    #retorna un diccionari de paraules i el seu recompte de vagades que ha sortit cada paraula.
    #
    #La funció de "TfidfTransformer" transforma el diccionari / recompte de la funció "CountVectorizer"
    #en una representació normalitzada tf (frequéncia) o tf-idf (frequéncia multiplicada per la inversa de la frequéncia),
    #aixó fa que obtinguem la frequéncia d'aparició d'una paraula.
    #
    #Finalment, tenim la funció "MultinomialNB" que és el classificador multinomial Naive Bayes, s'utilitza aquest perque 
    #és adequat per a la classificació amb característiques discretes (p. ex., recompte de paraules per a la classificació 
    #de text)
    
    naiveBayes = Pipeline([('countVect', CountVectorizer()),
                          ('tfidfTransf', TfidfTransformer()),
                          ('clf', MultinomialNB()),
    ])
    
    #entrenem
    naiveBayes = text_clf.fit(x_train, y_train)

    #test
    predicted = naiveBayes.predict(x_test)
    print ("Correct classification Naive Bayes withought stopwords      ", part, "% of the data: ", np.mean(predicted == y_test))


Correct classification Naive Bayes withought stopwords       0.1 % of the data:  0.793252436109579
Correct classification Naive Bayes withought stopwords       0.2 % of the data:  0.7942541944380602
Correct classification Naive Bayes withought stopwords       0.4 % of the data:  0.7823948517582165


Com s'observa, un cop eliminats els "stopwords" notem un molt petit increment en el porcentatge d'acerts que té el classificador, passant d'un 78,75% a un 79,42%. Aquesta millora és mínima i apenes notoria.

Ara anem a entrenar un classificador SVM (Support Vector Machines ) i el compararem els resultats amb el de Naive Bayes.

In [47]:
from sklearn.linear_model import SGDClassifier

X = dataset['title']
Y = dataset['topic'] #objectiu

particions = [0.1, 0.2, 0.4] #porcentatge de dades que s'utilitzaran en test

for part in particions:
    # Dividim dades en part d'entrenament i test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=part, random_state=0)

    #Creem un pipeline, que serveix per aplicar seqüencialment una llista de transformades i un 
    #estimador final (l'estimador final ha de portar implementat "fit").
    #
    #La funció de "CountVectorizer" s'utilitza per convertir un text en un vector de recomptes, es a dir, 
    #retorna un diccionari de paraules i el seu recompte de vagades que ha sortit cada paraula.
    #
    #La funció de "TfidfTransformer" transforma el diccionari / recompte de la funció "CountVectorizer"
    #en una representació normalitzada tf (frequéncia) o tf-idf (frequéncia multiplicada per la inversa de la frequéncia),
    #aixó fa que obtinguem la frequéncia d'aparició d'una paraula.
    #
    #Finalment, tenim la funció "MultinomialNB" que és el classificador multinomial Naive Bayes, s'utilitza aquest perque 
    #és adequat per a la classificació amb característiques discretes (p. ex., recompte de paraules per a la classificació 
    #de text)
    
    text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('svc', svm.SVC(C=0.001, kernel='linear', gamma=0.9, probability=True)),
    ])
    
    #entrenem
    svmMulti.fit(x_train, y_train)
    
    #test
    predicted_svm = svmMulti.predict(x_test)
    
    print ("Correct classification SVM with stopwords      ", part, "% of the data: ", np.mean(predicted_svm == y_test))
    

Correct classification SVM with stopwords       0.1 % of the data:  0.7297297297297297
Correct classification SVM with stopwords       0.2 % of the data:  0.7306366352562629
Correct classification SVM with stopwords       0.4 % of the data:  0.728430245920478


In [48]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm, datasets

X = dataTitleSW
Y = dataset['topic'] #objectiu

particions = [0.1, 0.2, 0.4] #porcentatge de dades que s'utilitzaran en test

for part in particions:
    # Dividim dades en part d'entrenament i test
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=part, random_state=0)

    #Creem un pipeline, que serveix per aplicar seqüencialment una llista de transformades i un 
    #estimador final (l'estimador final ha de portar implementat "fit").
    #
    #La funció de "CountVectorizer" s'utilitza per convertir un text en un vector de recomptes, es a dir, 
    #retorna un diccionari de paraules i el seu recompte de vagades que ha sortit cada paraula.
    #
    #La funció de "TfidfTransformer" transforma el diccionari / recompte de la funció "CountVectorizer"
    #en una representació normalitzada tf (frequéncia) o tf-idf (frequéncia multiplicada per la inversa de la frequéncia),
    #aixó fa que obtinguem la frequéncia d'aparició d'una paraula.
    #
    #Finalment, tenim la funció "MultinomialNB" que és el classificador multinomial Naive Bayes, s'utilitza aquest perque 
    #és adequat per a la classificació amb característiques discretes (p. ex., recompte de paraules per a la classificació 
    #de text)
    
    #svmMulti = Pipeline([('vect', CountVectorizer()),
     #                     ('tfidf', TfidfTransformer()),
     #                     ('clf-svm', SGDClassifier(alpha=0.01)),
     #])
    text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('svc', svm.SVC(C=0.001, kernel='linear', gamma=0.9, probability=True)),
    ])

    #entrenem
    svmMulti.fit(x_train, y_train)
    
    #test
    predicted_svm = svmMulti.predict(x_test)
    
    print ("Correct classification SVM withought stopwords      ", part, "% of the data: ", np.mean(predicted_svm == y_test))
    

Correct classification SVM withought stopwords       0.1 % of the data:  0.7336826622540908
Correct classification SVM withought stopwords       0.2 % of the data:  0.7331188232590209
Correct classification SVM withought stopwords       0.4 % of the data:  0.7315789473684211


In [ ]:

#print ("Correct classification SVM: ", svc.score(x_test, y_test))



from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('svc', svm.SVC(C=0.01, kernel='linear', gamma=0.9, probability=True)),
])
text_clf = text_clf.fit(x_train, y_train)

predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)